In [155]:
res = {}
res_path = Path("./results")
for _dir in res_path.iterdir():
    res[_dir.name] = {}
    for file in _dir.iterdir():
        one_res = joblib.load(file)
        res[_dir.name][file.name] = max(one_res.best_trial.intermediate_values.values())


In [183]:
df = pd.DataFrame.from_dict(res)
df.max(axis=1)

lemmas+no_stopwords.pkl          0.865316
clean+lemmas+no_stopwords.pkl    0.872467
clean+no_stopwords.pkl           0.871275
no_stopwords.pkl                 0.872467
clean.pkl                        0.874851
lemmas.pkl                       0.868892
clean+lemmas.pkl                 0.874851
dtype: float64